In [1]:
import cv2
import numpy as np
from pdf2image import convert_from_path
import pytesseract
import matplotlib.pyplot as plt
from PIL import Image
import PIL.ImageOps
import re
import csv
import os
import io
import pandas as pd
import openpyxl
import sys
from scipy.ndimage import interpolation as inter

In [3]:
xls = pd.ExcelFile('InfoDocSolicitud.xlsx')
df = pd.read_excel(xls, 'Hoja2')
df.head()


,Unnamed: 0,Id muestra,Nombre original,Comentarios,Tipo de documento,Apellido 1,Apellido 2,Nombre,NIF o NIE,Dirección,...,Apellidos.9,NIF o NIE.10,Nombre .10,Apellidos.10,NIF o NIE.11,Lugar,Día,Mes,Año,Unnamed: 69
0,1,Limpio_PDF_Muestra1,Daniel R.R.-solicitud-bono-social FIRMADO,"Docuemnto, A máquina, Limpio",1.0,REIZABAL,RIAÑO,DANIEL,25717784N,"ZAFIRO, 4",...,NaN,NaN,NaN,NaN,NaN,ARROYO de la MIEL,26.0,DICIEMBRE,2019.0,1
1,2,Limpio_PDF_Muestra2,Angel F Frechoso-Solicitud Bono Social.pdf,"Documento, A máquina, Limpio",1.0,MARTIN,FERNANDEZ,MARIA FERNANDA,17212935B,C/DIPUTADOS Nº1 PISO 7ºA,...,NaN,NaN,NaN,NaN,NaN,ZARAGOZA,15.0,ENERO,2020.0,2
2,3,LImpio_PDF_Muestra3,DANIEL GIMENO-solicitud bono social 2020,"Documento, A máquina, Limpio",1.0,GIMENO,GANDUL,DANIEL VICENTE,25146419J,MORIONES 6-10 ESC 3 1 A,...,NaN,NaN,NaN,NaN,NaN,ZARAGOZA,2.0,ENERO,2020.0,3
3,4,Medio_PDF_Muestra4,daroca-SOLICITUD RENOVACION BONO SOCIAL,"Escaneado, Limpio, A mano",1.0,BALLESTIN,GALAZARZA,ROBERTO,73096205C,CEBOLLADA 12,...,NaN,NaN,NaN,NaN,NaN,DAROCA,31.0,Enero,2020.0,4
4,5,Dificil_PDF_Muestra5,David Alessandra Girona-solicitud energia xxl114,"Escaneado, Tipex, Tachones, A mano, Atencion ...",1.0,GIRONA,ACUÑA,DAVID,44415840B,C/ DEL ROSSIGNOL Nº48,...,NaN,NaN,NaN,NaN,NaN,Tarragona,30.0,Enero,2020.0,5


In [15]:
for i in range(20,25):
    datos = []
    nombre_parcial = df.iloc[i,2]
    datos.append(df.iloc[i,5])
    datos.append(df.iloc[i,6])
    datos.append(df.iloc[i,7])
    datos.append(df.iloc[i,8])
    datos.append(df.iloc[i,9])
    datos.append(df.iloc[i,10])
    datos.append(df.iloc[i,12])
    datos.append(int(df.iloc[i,11]))
    datos.append(df.iloc[i,13])
    try:
        datos.append(int(df.iloc[i,15]))
    except ValueError:
        datos.append("")
    datos.append(int(df.iloc[i,14]))
    datos.append(df.iloc[i,16])
    nombre_doc = ".\\cosasAEntrenar\\" + nombre_parcial
    

            
    for i in range(12):
        with open(nombre_doc+"_"+str(i+1)+".gt.txt","w",encoding='UTF-8') as f:
            f.write(str(datos[i]))

    caracteres = {"├Б":"Á","├║":"ú","├│":"ó","├н":"í","├й":"é","├б":"á","o╠Б" :"ó"}
    for char in caracteres.keys():
        if char in nombre_doc:
            nombre_doc=nombre_doc.replace(char,caracteres[char])

## Extracción de imagenes

In [2]:
#buscamos en el excel
def encontrar(name, path):
    for root, dirs, files in os.walk(path):
        if name in files:
            return os.path.join(root, name)
        else:
            return 1
for i in range(20,25):
    nombre = df.iloc[i,2]
    nombre_doc = ".\\cosasAEntrenar\\" + nombre
    nombre_buscar = nombre + ".pdf"
    salida = encontrar(nombre_buscar, ".\\Conclusiones_OCR\\DocsJunto\\")
    if salida == 1:
        print("No encontrado")
    else: 
        imagenes = extract(salida)
        imagenes = imagenes[0:12]
        for j in range(len(imagenes)):
            cv2.imwrite(nombre_doc+"_"+str(j+1)+'.png',imagenes[j])
            
            

NameError: name 'df' is not defined

In [3]:
# Convertir de pdf a png
def convert_pdf_png(path,tit='test'):
    pages = convert_from_path(path, 500) #Subiendo este número hay que decir que tarda más pero lee mejor (más píxeles)
    i=1
    for page in pages:
        titulo=tit+str(i)+".png"
        page.save(titulo,"PNG")
        i=i+1
        break
        
        
# Mostrar imagen    
def mostrar(imagen):
    plt.figure(figsize=(20,15))
    plt.imshow(imagen)
    plt.title('Prueba bono')
    plt.show()

# Convertir a rgb de bgr
def rgb(imagen):
    b,g,r = cv2.split(imagen)
    rgb_img = cv2.merge([r,g,b])
    return rgb_img

# Invertir
def invertir(imagen):
    image = Image.open(imagen)
    inverted_image = PIL.ImageOps.invert(image)
    return inverted_image

# Extraer contornos, mostrarlos y OCR

def ajusta_perspectiva(path):
    img1 = cv2.imread(path)
    img2 = cv2.imread(r'limpia1.png')
    pts1=np.float32([[0,0],[img1.shape[1],0],[0,img1.shape[0]],[img1.shape[1],img1.shape[0]]])
    pts2=np.float32([[0,0],[img2.shape[1],0],[0,img2.shape[0]],[img2.shape[1],img2.shape[0]]])
    M=cv2.getPerspectiveTransform(pts1,pts2)
    img1=cv2.warpPerspective(img1,M,(img2.shape[1],img2.shape[0]))
    return img1
    

In [4]:
def extraer_contornos(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.blur(gray,(3,3)) 
    canny = cv2.Canny(gray,150,200)
    canny = cv2.dilate(canny, None, iterations=1)
    cnts = cv2.findContours(canny, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0]
    cnts = sorted(cnts, key=cv2.boundingRect, reverse=True)
    
    # Extrae los contornos en los que hay información (lista de diccionarios con las coordenadas)
    #Hay contornos que se cuelan por tamaño pero eso lo podemos solucionar por coordenadas, es decir, 
    #si por ejemplo está por encima de un valor de y no lo cogemos porque será de la parte que no hay info
    contornos= []
    for c in cnts:
        area = cv2.contourArea(c)
        if area > 45000:
            medidas={}
            x,y,w,h = cv2.boundingRect(c)
            if y>800:
                if h>250:
                    if h>350:
                        if h>480:
                            medidas['x']=x
                            medidas['y']=y
                            medidas['w']=w
                            medidas['h']=int(h/4)
                            contornos.append(medidas)

                            medidas['x']=x
                            medidas['y']=y+int(h/4)
                            medidas['w']=w
                            medidas['h']=int(h/4)
                            contornos.append(medidas)

                            medidas['x']=x
                            medidas['y']=y+int(2*h/4)
                            medidas['w']=w
                            medidas['h']=int(h/4)
                            contornos.append(medidas)

                            medidas['x']=x
                            medidas['y']=y+int(3*h/4)
                            medidas['w']=w
                            medidas['h']=int(h/4)
                            contornos.append(medidas)
                        else:
                            medidas['x']=x
                            medidas['y']=y
                            medidas['w']=w
                            medidas['h']=int(h/3)
                            contornos.append(medidas)

                            medidas['x']=x
                            medidas['y']=y+int(h/3)
                            medidas['w']=w
                            medidas['h']=int(h/3)
                            contornos.append(medidas)

                            medidas['x']=x
                            medidas['y']=y+int(2*h/3)
                            medidas['w']=w
                            medidas['h']=int(h/3)
                            contornos.append(medidas)
                    else:
                        medidas['x']=x
                        medidas['y']=y
                        medidas['w']=w
                        medidas['h']=int(h/2)
                        contornos.append(medidas)

                        medidas['x']=x
                        medidas['y']=y+int(h/2)
                        medidas['w']=w
                        medidas['h']=int(h/2)
                        contornos.append(medidas)
                else:
                    medidas['x']=x
                    medidas['y']=y
                    medidas['w']=w
                    medidas['h']=h
                    contornos.append(medidas)
    contornos=sorted(contornos, key=lambda k: k['x'])
    contornos=sorted(contornos, key=lambda k: k['y']) 
        
    print('\n')
    print("Numero total de contornos detectados:",len(contornos))
    print('\n')
    
    return contornos

In [5]:
def eliminar_lineas(image):
    # Tratar la imagen
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.bitwise_not(gray)
    bw = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, \
                                   cv2.THRESH_BINARY, 15, -5)

    # Crear copy sobre las que se haran las lineas horizontales y verticales
    horizontal = np.copy(bw)
    vertical = np.copy(bw)
    
    # ----- H o r i z o n t a l e s -----
    cols = horizontal.shape[1]
    horizontal_size = cols // 30 #tamaño mínimo de las líneas a eliminar
    #Creamos un elemento que extrae las líneas horizontales a través de operaciones morfológicas
    horizontalStructure = cv2.getStructuringElement(cv2.MORPH_RECT, (horizontal_size, 1))

    # Aplicamos las operaciones morfológicas
    horizontal = cv2.erode(horizontal, horizontalStructure)
    horizontal = cv2.dilate(horizontal, horizontalStructure)
    
    #restamos al documento original las líneas horizontales detectadas
    resta=cv2.absdiff(bw,horizontal)
    

    # ----- V e r t i c a l e s -----
    rows = vertical.shape[0]
    verticalsize=rows//65
    #Creamos un elemento que extrae las líneas horizontales a través de operaciones morfológicas
    verticalStructure = cv2.getStructuringElement(cv2.MORPH_RECT, (1, verticalsize))
    
    # Aplicamos las operaciones morfológicas
    vertical = cv2.erode(vertical, verticalStructure)
    vertical = cv2.dilate(vertical, verticalStructure)
    #mostrar(vertical)
    
    # ----- R e s t a -----
    #restamos al documento anterior que obtuvimos después del borrado de las horizontales, las verticales
    resta_final=cv2.absdiff(resta,vertical)
    
    #print("Imagen sin cuadricula:")
    cv2.imwrite("restadoc.png",resta_final)
    
    invertida=cv2.bitwise_not(resta_final)
    cv2.imwrite("invertida.png",invertida)
    
    #primero quitando ruido y luego invirtiendo, en realidad los imwrite me sobran, pero es para ver las imágenes
    #Buscar una forma de quitar el ruido más robusta, xq depende un poco del doc que analicemos
    #Por ejemplo, hay algunos de a mano que con 9 van bien, otros que a ordenador van bien con 7 y otros que con 5
    sinruido = cv2.medianBlur(invertida,5) #7 o 9??
    cv2.imwrite("sinruido.png",sinruido)
    sinruido = cv2.imread("sinruido.png") #La volvemos a leer porque se visualiza mejor después
    
    return sinruido

In [6]:
def extraer():
    image = ajusta_perspectiva(r'test1.png')
    #mostrar(image) Cada vez que muestra tarda bastante más
    image=rgb(image)
    #print("Imagen original:")
    #mostrar(image)
    # Config imagen
    contornos = extraer_contornos(image)

##----------------------    D e t e c t a r    l i n e a s  -----------------------
    sinlineas = eliminar_lineas(image)
    #mostrar(resta_final)
    # ------ A n a l i s i s   d e   c o n t o r n o s
    for i in range(len(contornos)):
        x=contornos[i]['x']
        y=contornos[i]['y']
        w=contornos[i]['w']
        h=contornos[i]['h']
        trozo=sinlineas[y:y+h,x:x+w]
        trozo=cv2.medianBlur(trozo,7)
        nombre_parcial
        cv2.imwrite(os.join.path(".\\cosasAEntrenar",nombre_parcial),trozo)
        #mostrar(trozo)
        #print("Texto detectado:")
        #print(pytesseract.image_to_string(trozo, config=custom_config))


In [7]:
def lee_tesseract(doc,contornos):
    resultados=[]
    for i in range(len(contornos)):
        x=contornos[i]['x']
        y=contornos[i]['y']
        w=contornos[i]['w']
        h=contornos[i]['h']
        trozo=doc[y:y+h,x:x+w]
        trozo=cv2.medianBlur(trozo,7)
        resultados.append(trozo)
    return resultados

def extract(path_doc):
    convert_pdf_png(path_doc)
    image = ajusta_perspectiva(r'test1.png') 
    image=rgb(image)
    contornos = extraer_contornos(image)
    sinlineas = eliminar_lineas(image)
    leido = lee_tesseract(sinlineas,contornos)
    return leido
    


In [8]:
pytesseract.pytesseract.tesseract_cmd = r'C:\\Program Files\\Tesseract-OCR'



Numero total de contornos detectados: 16




[array([[[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],
 
        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],
 
        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],
 
        ...,
 
        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],
 
        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],
 
        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
  

In [25]:
##queremos limpiar los espacios
for f in os.listdir(".\\cosasAEntrenar\\"):
    r = f.replace(" ","")
    if(r != f):
        os.rename(f,r)

FileNotFoundError: [WinError 2] El sistema no puede encontrar el archivo especificado: 'Cristóbal Molina-1.solicitud_1.gt.txt' -> 'CristóbalMolina-1.solicitud_1.gt.txt'